1. you have to create one supervisor node.
2. create one router function
3. create three more node
3.1 llm call (llm node)
3.2 RAG (rag node)
3.3 web crawler(fetch the info in realtime from internet)
4. created one more node after this for validation for generated output --> explore the validation part how to do that
5. if validation going to be failed in that case again go to supervioser node and then supervisor node will again decide what needs to be call next
6. once the validation will pass then only generate the final output

submission deadline till 9pm friday

submission instruction:
create your github repo and keep all the assisgnments over there(in that github repo)
i will share one googleform in group after completing your assignment you can share the github link through that google form.

In [ ]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END
from dotenv import load_dotenv
from langchain.output_parsers import PydanticOutputParser
import os
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_tavily import TavilySearch

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=OPENAI_API_KEY
)

# Create Vector Retriever

In [2]:
loader=DirectoryLoader("data",glob="./*.txt",loader_cls=TextLoader)
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
docs_splitted = splitter.split_documents(documents=docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(docs_splitted, embeddings)

In [3]:
retriever = db.as_retriever(search_kwargs={"k": 3})

# Agent State

In [44]:
class State(TypedDict):
    question: str
    usa: bool
    chat: bool
    web: bool
    response: str
    is_validated: bool
    final_response: str
    feedback: str
    reason: str


# Supervisor(Router)

In [45]:
class RouterOutput(BaseModel):
    usa: bool=Field(description="If the question is related to USA Economy")
    chat: bool=Field(description="If the question is a normal chat conversation")
    web: bool=Field(description="If the question is neither USA economy or chat conversation")

router_parser = PydanticOutputParser(pydantic_object=RouterOutput)

In [46]:
def router_node(state:State):

    question = state["question"]

    if state.get("feedback"):
        template = """
        You are a orchestrator. You will receive a question and your job is to route this question into 3 agents:
        1) Chat Agent - Route to this agent when the question is a normal chat conversation
        2) RAG Agent - Route to this agent when the question is related to United States economy
        3) Web Search Agent - Route to this agent when the question is not related to a chat conversation or USA Economy.

        Question: \n
        {question} \n\n

        The final answer will be validated by another agent. If the answer was rejected, you will receive a feedback to improve it.

        Feedback: \n
        {feedback} 
        
        {format_instructions}
        """

        prompt = PromptTemplate(
            template=template,
            input_variables=["question","feedback"],
            partial_variables={
                "format_instructions": router_parser.get_format_instructions(),
                }
        )
    else:
        template = """
        You are a orchestrator. You will receive a question and your job is to route this question into 3 agents:
        1) Chat Agent - Route to this agent when the question is a normal chat conversation
        2) RAG Agent - Route to this agent when the question is related to United States economy
        3) Web Search Agent - Route to this agent when the question is not related to a chat conversation or USA Economy.

        Question: \n
        {question} \n\n 
        
        {format_instructions}
        """

        prompt = PromptTemplate(
            template=template,
            input_variables=["question"],
            partial_variables={
                "format_instructions": router_parser.get_format_instructions(),
                }
        )

    chain = prompt | llm | router_parser

    response = chain.invoke({"question":question})

    print(f"Router Decision: RAG: {response.usa} Web: {response.web} Chat: {response.chat}")

    return {"usa": response.usa,
            "web": response.web,
            "chat": response.chat
            }

In [47]:
def router_condition(state:State):
    if state['usa'] == True:
        return "usa"
    elif state['web'] == True:
        return "web"
    elif state['chat'] == True:
        return "chat"
    else:
        return "chat"

# Rag Node

In [48]:
def format_docs(docs):
    return "/n/n".join(doc.page_content for doc in docs)

In [49]:
def rag_node(state:State):
    print("\n----- STARTING RAG NODE -----")
    question = state["question"]

    template = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    
    \nQuestion: {question} \nContext: {context} \nAnswer:
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["question","context"]
    )

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    ) 

    response = rag_chain.invoke(question)

    return {"response":response}

    

# Chat(not related node)

In [50]:
def chat_node(state:State):
    print("\n----- STARTING Chat NODE -----")

    question = state['question']

    template = """
    You are a helpful assistant. Answer the user question in a helpful way.

    Question: /n
    {question}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["question"]
    )

    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"question":question})

    return {"response":response}

# Web Search Node

In [51]:
def web_search_node(state:State):
    print("\n----- STARTING WEB NODE -----")

    question = state['question']

    tool = TavilySearch(
    max_results=3,
    topic="general",
)
    web_results = tool.invoke(question)
    print(f"WEB Results: {web_results}")

    template = """
    You are a helpful assistant. Use the Web Search results and the user's question to answer accordingly.

    \nQuestion:
    {question}

    \nWeb Search Results:
    {web_results}
    """   

    prompt = PromptTemplate(
        template=template,
        input_variables=["question", "web_results"]
    )

    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"question":question, "web_results":web_results})

    return {"response":response}

# Validation Node

In [52]:
class ValidationParser(BaseModel):
    is_validated: bool=Field(description="If the answer is validated based on the user's question.")
    feedback: str=Field(description="The reason why the answer was not validated.")

validation_parser = PydanticOutputParser(pydantic_object=ValidationParser)

In [58]:
def validation_node(state:State):
    print("\n----- STARTING VALIDATION NODE -----")

    question = state['question']

    response = state['response']

    template = """
    You are a validation agent. You will receive a question from the user and an answer given by another agent.
    Your job is to validate if the answer is related to the question. You don't need to validate if the answer is correct or not.
    If the answer is related to the question, return a true validation. If not, return a false validation and a feedback on how to improve

    \nExamples:
    question: Which's the biggest country on planet?
    answer: Unicorn
    is_validated = False

    question: How much was the Germany GDP in 2024?
    answer: 30 Trillion
    is_validated = True


    \nQuestion:
    {question}

    \nAnswer: 
    {response}

    \n{format_instructions}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["question", "response"],
        partial_variables={"format_instructions": validation_parser.get_format_instructions()}
    )

    chain = prompt | llm | validation_parser

    print(f"Question: {question} \n Answer: {response}")
    response = chain.invoke({"question":question, "response":response})
    print(f"Validation Results: Is Validated: {response.is_validated} Feedback: {response.feedback}")
    

    return {"is_validated":response.is_validated, "feeback":response.feedback}

In [59]:
def validation_condition(state:State):
    if state['is_validated'] == True:
        return "is_validated"
    else:
        return "not_validated"

# Final Answer

In [60]:
def output_final_answer(state:State):
    print("\n----- STARTING OUTPUT FINAL ANSWER NODE -----")

    if state['is_validated'] == True:
        final_answer = state['response']
        return {"final_answer":final_answer}

# Workflow

In [61]:
workflow = StateGraph(State)
workflow.add_node("router", router_node)
workflow.add_node("rag_node",rag_node)
workflow.add_node("chat_node",chat_node)
workflow.add_node("web_search_node",web_search_node)
workflow.add_node("validation_node",validation_node)
workflow.add_node("final_answer", output_final_answer)

workflow.add_edge("rag_node", "validation_node")
workflow.add_edge("web_search_node", "validation_node")
workflow.add_edge("chat_node", "validation_node")


workflow.set_entry_point("router")
workflow.set_finish_point("final_answer")

workflow.add_conditional_edges(
    "router",
    router_condition,
    {
        "usa": "rag_node",
        "chat": "chat_node",
        "web": "web_search_node"
    }
)

workflow.add_conditional_edges(
    "validation_node",
    validation_condition,
    {
        "is_validated":"final_answer",
        "not_validated":"router"
    }
)

graph = workflow.compile()

In [24]:
graph.invoke({"question":"What's the biggest animal on earth?"})

Router Decision: RAG: False Web: False Chat: True

----- STARTING Chat NODE -----

----- STARTING VALIDATION NODE -----
Validation Results: Is Validated: True Feedback: 

----- STARTING OUTPUT FINAL ANSWER NODE -----


{'question': "What's the biggest animal on earth?",
 'usa': False,
 'chat': True,
 'web': False,
 'response': 'The biggest animal on Earth is the blue whale. Blue whales can reach lengths of up to 98 feet (30 meters) and weigh as much as 200 tons or more. They are the largest animals known to have ever existed on our planet.',
 'is_validated': True}

In [18]:
graph.invoke({"question":"How much is the GDP of the USA?"})


Router Decision: RAG: True Web: False Chat: False

----- STARTING RAG NODE -----

----- STARTING VALIDATION NODE -----
Validation Results: Is Validated: True Feedback: 

----- STARTING OUTPUT FINAL ANSWER NODE -----


{'question': 'How much is the GDP of the USA?',
 'usa': True,
 'chat': False,
 'web': False,
 'response': 'The GDP of the USA is estimated to be around $28 trillion USD as of 2024. It is the largest in the world by nominal GDP.',
 'is_validated': True}

In [40]:
graph.invoke({"question":"What's the price of bitcoin today(10th june 2025)"})


Router Decision: RAG: False Web: True Chat: False

----- STARTING WEB NODE -----
WEB Results: {'query': "What's the price of bitcoin today(10th june 2025)", 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Bitcoin Price Today, June 10, 2025: BTC Price Dips to USD ... - LatestLY', 'url': 'https://www.latestly.com/socially/technology/bitcoin-price-today-june-10-2025-btc-price-dips-to-usd-109227-63-after-surpassing-usd-110000-mark-6917982.html', 'content': 'Bitcoin price dipped to USD 109,227.63 after nearing its all-time high earlier that day on June 10, 2025. Despite the fluctuation in BTC price, analysts remain optimistic, projecting a potential recovery to USD 1,20,000 by the end of 2025. 📲 Bitcoin Price Today, June 10, 2025: BTC Price Dips to USD 109,227.63 After Surpassing USD 1,10,000 Mark.', 'score': 0.94632465, 'raw_content': None}, {'title': 'Crypto Price Today (June 10): Ethereum Briefly Touches $2,800; Bitcoin ...', 'url': 'https://www.cryptoti

{'question': "What's the price of bitcoin today(10th june 2025)",
 'usa': False,
 'chat': False,
 'web': True,
 'response': 'As of June 10, 2025, the price of Bitcoin is approximately USD 109,227.63.',
 'is_validated': True}

In [ ]:
graph.invoke({"question":"Hello?"})


Router Decision: RAG: False Web: False Chat: True

----- STARTING Chat NODE -----

----- STARTING VALIDATION NODE -----
Question: Hello? 
 Answer: Hello! How can I assist you today?
Validation Results: Is Validated: True Feedback: The answer is a polite and relevant response to a greeting question.

----- STARTING OUTPUT FINAL ANSWER NODE -----


{'question': 'Hello?',
 'usa': False,
 'chat': True,
 'web': False,
 'response': 'Hello! How can I assist you today?',
 'is_validated': True}

In [63]:
graph.invoke({"question":"how to create an unicorn?"})


Router Decision: RAG: False Web: False Chat: True

----- STARTING Chat NODE -----

----- STARTING VALIDATION NODE -----
Question: how to create an unicorn? 
 Answer: Creating a real unicorn, as depicted in myths and legends, is not possible since unicorns are mythical creatures. However, if you're interested in creating a unicorn-themed craft or decoration, here are some fun ideas:

1. **Unicorn Crafting with Paper or Clay:**
   - Use white, pink, and rainbow-colored paper or clay to make a unicorn figure.
   - Add a glittery horn, mane, and tail for a magical touch.

2. **Decorating a Unicorn Theme Party:**
   - Decorate with unicorn balloons, banners, and plush toys.
   - Use pastel and rainbow colors for a whimsical atmosphere.

3. **DIY Unicorn Horn:**
   - Make a horn using rolled paper or foam cone, decorate with glitter, and attach it to a headband for a costume piece.

If you're interested in storytelling or artistic projects, you could also write a story about creating your ow

{'question': 'how to create an unicorn?',
 'usa': False,
 'chat': True,
 'web': False,
 'response': "Creating a real unicorn, as depicted in myths and legends, is not possible since unicorns are mythical creatures. However, if you're interested in creating a unicorn-themed craft or decoration, here are some fun ideas:\n\n1. **Unicorn Crafting with Paper or Clay:**\n   - Use white, pink, and rainbow-colored paper or clay to make a unicorn figure.\n   - Add a glittery horn, mane, and tail for a magical touch.\n\n2. **Decorating a Unicorn Theme Party:**\n   - Decorate with unicorn balloons, banners, and plush toys.\n   - Use pastel and rainbow colors for a whimsical atmosphere.\n\n3. **DIY Unicorn Horn:**\n   - Make a horn using rolled paper or foam cone, decorate with glitter, and attach it to a headband for a costume piece.\n\nIf you're interested in storytelling or artistic projects, you could also write a story about creating your own magical unicorn. Let me know if you'd like more id